### FAP tests

We are testing against the null hypothesis that the white dwarf found was a chance alignment of a random source that just happened to meet our criteria (i.e. appears co-moving with the pulsar, has properties consistent with being a WD) with the pulsar position on the sky.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from astropy import units as u
from astropy.coordinates import SkyCoord, Angle
from astropy.time import Time
from astropy.table import Table, unique
from tqdm import tqdm
from astroquery.gaia import Gaia
from scipy.special import erfinv, erfcinv, erf, erfc

In [2]:
pulsar_data = np.genfromtxt('ipta_3arcsec_wpm-result.csv', delimiter=',', names=True, dtype=None, encoding=None)

In [3]:
pulsar_data = Table(pulsar_data)
pulsar_data.sort(['names', 'angdist'])
pulsar_data = unique(pulsar_data, keys='names') # drop less-good duplicate matches

This function will reproduce the cuts that we made to determine which nearby sources were candidate matches and which should be thrown out:

**NOTE: absolute magnitude > 7 cut was made relatively arbitrarily to weed out main-sequence stars from white dwarfs - this may need to be changed depending on what criteria we settle on for the paper.**

In [4]:
def make_cuts(tbl, pulsar_pmra, pulsar_pmdec):
    # take an astropy table, apply cuts
    pmra_tol = np.abs(3 * np.abs(tbl['pmra_error'])) # 3 sigma
    pmdec_tol = np.abs(3 * np.abs(tbl['pmdec_error'])) # 3 sigma
    idx = np.isfinite(tbl['parallax']) & (np.abs(np.abs(tbl['pmra']) - np.abs(pulsar_pmra)) <= pmra_tol) \
           & (np.abs(np.abs(tbl['pmdec']) - np.abs(pulsar_pmdec)) <= pmdec_tol) 
#            & ((tbl['phot_g_mean_mag'] + 5.*np.log10(tbl['parallax']) - 10.) >= 5.) #no abs mag cut
    return tbl[idx]

In [5]:
make_cuts(pulsar_data, 
          pulsar_data['pmra_1'], 
          pulsar_data['pmdec_1'])['names']

J0437-4715
J1012+5307
J1024-0719
J1732-5049
J1747-4036
J1843-1113


Now we'll randomize each pulsar's coordinates (within 3 degrees in RA and Dec to reflect the actual crowding in the general sky region) and see how many objects in the new area pass our cuts.

In [11]:
pulsars_to_test = make_cuts(pulsar_data, 
          pulsar_data['pmra_1'], 
          pulsar_data['pmdec_1']) # just the good matches
faps = []
N_trials = 10000
search_radius = u.Quantity(3.0, u.arcsec)
p  = pulsars_to_test[3] # do one pulsar at a time
print('calculating FAP for pulsar {0}...'.format(p['names']))
n_found = np.zeros(N_trials)
for n in tqdm(range(N_trials)):
    test_ra = p['ra_1'] + np.random.normal(0., 3.) # randomized within 3 degrees
    test_dec = p['dec_1'] + np.random.normal(0., 3.) # randomized within 3 degrees
    test_coord = SkyCoord(ra=test_ra, dec=test_dec, unit=(u.degree, u.degree))
    j = Gaia.cone_search_async(test_coord, search_radius)
    r = j.get_results()
    n_found[n] = len(make_cuts(r, p['pmra_1'], p['pmdec_1']))
    print('found FAP of {0} in {1}.'.format(np.sum(n_found > 0), N_trials))
    f=open('../FAPs/FAP_J.txt','a+')
    ##f.write(n_found)
    np.savetxt(f, n_found)
    f.close()
faps.append(np.sum(n_found > 0)/N_trials)
    
    


  0%|          | 0/10000 [00:00<?, ?it/s]

calculating FAP for pulsar J1732-5049...


  0%|          | 1/10000 [00:02<7:31:11,  2.71s/it]

INFO: Query finished. [astroquery.utils.tap.core]
found FAP of 0 in 10000.


  0%|          | 1/10000 [00:03<9:44:14,  3.51s/it]


KeyboardInterrupt: 

In [8]:
for p,f in zip(pulsars_to_test, faps):
    print("{0}: FAP {1}".format(p['names'], f))

# FAP of J1843

In [7]:
foundfile1843 = np.loadtxt("../FAPs/FAP_J1843.txt")

In [8]:
FAP_1843 = np.sum(foundfile1843)/len(foundfile1843)

In [9]:
len(foundfile1843)

184660000

In [10]:
"%.2e"%FAP_1843

'6.99e-02'

In [11]:
print("This is a {} detection".format(erfcinv(FAP_1843)*np.sqrt(2)))

This is a 1.8124148036518855 detection


In [12]:
FAP_1843

0.06992212715260479

# FAP of J0437

In [49]:
foundfile0437 = np.loadtxt("../FAPs/FAP_J0437.txt")

In [50]:
print(len(foundfile0437))

134890000


In [51]:
FAP_0437 = np.sum(foundfile0437)/len(foundfile0437)
print(FAP_0437)

0.0


In [52]:
print("This is greater than a {} detection".format(erfcinv(1/len(foundfile0437))*np.sqrt(2)))

This is greater than a 5.781279779781753 detection


# FAP of J1012

In [6]:
foundfile1012 = np.loadtxt("../FAPs/FAP_J1012.txt")

In [7]:
print(len(foundfile1012))

107420000


In [8]:
FAP_1012 = np.sum(foundfile1012)/len(foundfile1012)
print(FAP_1012)

0.0004198007819772854


In [10]:
print("This is greater than a {} detection".format(erfcinv(FAP_1012)*np.sqrt(2)))

This is greater than a 3.5273126206054135 detection


# FAP of J1024

In [10]:
foundfile1024 = np.loadtxt("../FAPs/FAP_J1024.txt")

In [11]:
print(len(foundfile1024))

232765576


In [12]:
FAP_1024 = np.sum(foundfile1024)/len(foundfile1024)
print("FAP of J1024 is less than {0}".format(1/len(foundfile1024)))

FAP of J1024 is less than 4.296167917888339e-09


In [13]:
print("This is greater than a {} detection".format(erfcinv(FAP_1024))*np.sqrt(2))

This is greater than a 5.872366828730009 detection


In [11]:
erfcinv(4.296167917888339e-09)*np.sqrt(2)

5.872366828730009

# FAP of J1732

In [10]:
foundfile1732 = np.loadtxt("../FAPs/FAP_J1732 2.txt")

In [11]:
print(len(foundfile1732))

310100000


In [12]:
FAP_1732 = np.sum(foundfile1732)/len(foundfile1732)
print("%.2e"%FAP_1732)

2.18e-02


In [13]:
FAP_1732

0.021837852305707835

In [14]:
print("This is a {} detection".format(erfcinv(FAP_1732)*np.sqrt(2)))

This is a 2.2931763820794404 detection


# FAP of J1910

In [20]:
foundfile1910 = np.loadtxt("../FAPs/FAP_J1910.txt")

In [21]:
"%.2e"%len(foundfile1910)

'2.59e+08'

In [22]:
FAP_1910 = np.sum(foundfile1910)/len(foundfile1910)
print("%.2e"%FAP_1910)

6.44e-02


In [23]:
print("This is a {} detection".format(erfcinv(FAP_1910)*np.sqrt(2)))

This is a 1.8495296359264826 detection


In [24]:
print(len(foundfile1910))

259080000


# FAP of J1744

In [6]:
foundfile1744 = np.loadtxt("../FAPs/FAP_J1744.txt")

In [7]:
"%.2e"%len(foundfile1744)

'2.37e+08'

In [8]:
FAP_1744 = np.sum(foundfile1744)/len(foundfile1744)
print("%.2e"%FAP_1744)

4.30e-02


In [9]:
print("This is a {} detection".format(erfcinv(FAP_1744)*np.sqrt(2)))

This is a 2.0236247596858443 detection


In [10]:
print(len(foundfile1744))

236910000


# Pulsar J1747

In [7]:
foundfile1747 = np.loadtxt("../FAPs/FAP_J1747.txt")

In [9]:
print(len(foundfile1747))

151780000


In [11]:
FAP_J1747 = np.sum(foundfile1747)/len(foundfile1747)
print("%.2e"%FAP_J1747)

1.24e-01


In [13]:
print("This is a {} detection".format(erfcinv(FAP_J1747)*np.sqrt(2)))

This is a 1.536324008681274 detection
